In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
if not torch.cuda.is_available():
    print("GPU is not enabled")
else:
    print("GPU is enabled")

GPU is not enabled


In [3]:
import os

os.chdir('/Users/eamon/Desktop/AIM/')
os.getcwd()

'/Users/eamon/Desktop/AIM'

In [37]:
from deepfake_dataloader import DeepfakeDataset 

dataset = DeepfakeDataset(directory_path='../release_in_the_wild/', length=1250, transform=transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to the required input size of your CNN
    transforms.ToTensor(),           # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
]))

In [38]:
len(dataset)

1250

In [39]:
print(f'The shape of each instance is {dataset[0][0].shape}')

The shape of each instance is torch.Size([3, 224, 224])


In [40]:
print(f'The label of the first instance is {dataset[0][1]}')

The label of the first instance is tensor([1])


In [36]:
from torch.utils.data import random_split

train_dataset, test_dataset = random_split(dataset, [.8, .2])

batch_size = 16

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [46]:
print(f'The size of the train_dataloader is {len(dataset) * .8:.0f}')
print(f'The size of the test_dataloader is {len(dataset) * .2:.0f}')

The size of the train_dataloader is 1000
The size of the test_dataloader is 250


# Training the model

In [47]:
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18

In [49]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

class CustomResNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet18(weights=None)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.resnet(x)

model = CustomResNet(num_classes=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        labels = labels.squeeze(1)

        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')


Finished Training


In [50]:
model.eval()  

correct = 0
total = 0

with torch.no_grad():  
    for data in test_dataloader:
        inputs, labels = data
        labels = labels.squeeze(1)  
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)  
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test dataset: %d %%' % (
    100 * correct / total))


Accuracy of the network on the test dataset: 96 %


Accuracy looked way too good so I had to ask chatgpt to help find EER 

In [51]:
import torch
import torch.nn.functional as F

# Initialize an empty list to store the scores
scores = []

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation for evaluation
with torch.no_grad():
    for data in test_dataloader:
        inputs, _ = data  # Assuming you don't need labels for this
        outputs = model(inputs)
        
        # Apply softmax to convert logits to probabilities
        probabilities = F.softmax(outputs, dim=1)
        
        # Extract the probability of the positive class (class 1)
        positive_scores = probabilities[:, 1]
        
        # Convert to numpy array and append to the scores list
        scores.extend(positive_scores.cpu().numpy())

# Convert scores list to numpy array
scores = np.array(scores)

# Now you have the array of confidence scores from the model
print(scores)


[5.79836905e-01 9.99817073e-01 3.41197327e-02 9.99971271e-01
 6.89652516e-03 2.91959226e-01 9.87306356e-01 9.98737276e-01
 5.37002325e-01 1.07502136e-02 9.27562546e-03 2.90082037e-01
 9.93652225e-01 1.45574128e-02 9.70215678e-01 4.21678126e-02
 1.48974890e-02 4.21220378e-04 1.00150276e-02 9.30585563e-01
 1.85425917e-03 7.23335310e-04 8.71605182e-04 9.97721255e-01
 9.94293988e-01 9.99912858e-01 7.31554767e-03 7.45700393e-03
 2.06408463e-03 9.84534979e-01 1.72656099e-03 3.66550148e-03
 1.49135347e-02 9.99938250e-01 1.54276267e-01 5.69233613e-04
 9.99714315e-01 4.34447732e-03 9.95333374e-01 3.76384170e-03
 2.37905756e-02 7.42593734e-03 4.20900574e-03 3.20174336e-01
 2.17483169e-03 4.39920649e-03 1.85840186e-02 1.12543325e-03
 1.52133405e-02 1.45362355e-02 1.16604771e-02 1.07079349e-01
 1.34519646e-02 9.96878982e-01 9.72107111e-04 5.37815364e-03
 5.26700556e-01 4.14094393e-04 1.27599929e-02 4.61189657e-01
 9.60543931e-01 9.99980927e-01 9.99509096e-01 2.48742639e-03
 1.50615862e-03 9.969328

In [52]:
scores.shape

(240,)

In [53]:
import numpy as np

# Initialize an empty list to store the true labels
true_labels = []

# Iterate over the test dataset using the test_dataloader
for data in test_dataloader:
    _, labels = data  # Assuming labels are provided in the second element of the tuple
    labels = labels.squeeze(1)  # Remove extra dimension if present
    true_labels.extend(labels.cpu().numpy())

# Convert true labels list to numpy array
true_labels = np.array(true_labels)

# Now you have the true labels from the test dataset
print(true_labels)


[1 1 0 1 0 0 1 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 1 0 1 0
 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 1 1 0 1 0 0 1 0 1 1 0 1 1 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 0 1 0
 0 1 0 1 1 1 0 1 0 0 1 0 0 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 1 0
 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0]


In [65]:
print(scores[:7])
print(true_labels[:7])

[0.5798369  0.9998171  0.03411973 0.9999713  0.00689653 0.29195923
 0.98730636]
[1 1 0 1 0 0 1]


In [68]:
import numpy as np
from sklearn.metrics import roc_curve

# Assuming you have the scores and true labels
# scores: array of confidence scores from your model
# labels: true labels (0 or 1)

# Compute the ROC curve
fpr, tpr, thresholds = roc_curve(true_labels, scores)

# Calculate the FAR and FRR
far = 1 - tpr
frr = fpr

# Find the point where FAR equals FRR
eer_threshold = thresholds[np.nanargmin(np.abs(far - frr))]
eer = (far[np.nanargmin(np.abs(far - frr))] + frr[np.nanargmin(np.abs(far - frr))]) / 2

print("Equal Error Rate (EER): {:.2f}%".format(eer * 100))


Equal Error Rate (EER): 4.98%
